In [4]:
from __future__ import print_function
import xml.etree.ElementTree as ET
import os
import sys
import nltk
from nltk.parse.corenlp import CoreNLPDependencyParser

SyntaxError: invalid syntax (probability.py, line 333)

In [2]:

import xml.etree.ElementTree as ET
import os
import sys
import nltk
from nltk.parse.corenlp import CoreNLPDependencyParser

# TODO see if is used
nltk.download('punkt')

class Token:
    def __init__(self, word, offset_from, offset_to):
        self.word = word
        self.offset_from = int(offset_from)
        self.offset_to = int(offset_to)
        self.type = "O"  # Initialize all tokens with type O as non-drugs non-brands and non-groups

    def word_iscapitalized(self):
        return self.word[0].isupper()

    def word_isupper(self):
        return self.word.isupper()


class Entity:

    def __init__(self, **kwargs):
        self.word = kwargs["text"]
        self.offset_from, self.offset_to = self.parse_offset(kwargs["charOffset"])
        self.type = kwargs["type"]

    def parse_offset(self, offset):

        # offset can be given in two ways
        # e.g.:
        #       * 9-23
        #       * 9-11;12-20;21-23
        #
        # We differenciate both cases and always save the first one and the last one

        if ";" in offset:
            offset = offset.split(";")
            offset_from = offset[0].split('-')[0]
            offset_to = offset[-1].split('-')[1]
        else:
            offset_from, offset_to = offset.split('-')

        return int(offset_from), int(offset_to)


# Store as a list of dictionaries the word, the offset interval and the label (drug, group, brand, drug_n,...) of each entity in the sentence.
def get_entities(sentence):
    entities = []
    for ent in sentence.findall('entity'):
        entity = Entity(**ent.attrib)
        entities.append(entity)
    return entities

def tokenize(sentence, entities):
    span_generator = WhitespaceTokenizer().span_tokenize(sentence)
    tokens = [(sentence[span[0]: span[1]], span[0], span[1] - 1) for span in span_generator]

    new_tokens = []
    for i, token in enumerate(tokens):
        word, offset_from, offset_to = token

        if (len(word) > 1) and (word.endswith(',') or word.endswith('.') or word.endswith(':') or word.endswith(';')):
            punct = word[-1]
            punct_offset_from = offset_to
            punct_offset_to = offset_to

            word = word[:-1]
            offset_to -= 1

            new_tokens.append(Token(word, offset_from, offset_to))
            new_tokens.append(Token(punct, punct_offset_from, punct_offset_to))

        elif (len(word) > 1) and word[0] == '(' and (word[0:2] != '(+' or word[0:2] != '(-'):
            punct = word[0]
            punct_offset_from = offset_from
            punct_offset_to = offset_from

            word = word[1:]
            offset_from += 1

            new_tokens.append(Token(punct, punct_offset_from, punct_offset_to))
            new_tokens.append(Token(word, offset_from, offset_to))
        else:
            new_tokens.append(Token(word, offset_from, offset_to))

    return new_tokens

def detect_label(token, entities):
    for entity in entities:
        # If the two offsets are equal, then it corresponds to the same word and type.
        if token.offset_from == entity.offset_from and token.offset_to == entity.offset_to:
            token.type = "B-" + entity.type
            return
        # If the token offset interval is inside the entity offset interval, then it is a first or the continuation of a type sequence.
        elif entity.offset_from <= token.offset_from and token.offset_to <= entity.offset_to:
            if entity.offset_from == token.offset_from:
                token.type = "B-" + entity.type
            else:
                token.type = "I-" + entity.type

            return
        else:
            token.type = "O"

def load_data(datadir):
    #TODO
    # returns dataset as a dict of examples: where the keys are the identities and the value is the tokenization list of the entity.

    dataset_dict = {}
    for filename in os.listdir(datadir):
        # parse XML file, obtaining a DOM tree
        file_path = os.path.join(datadir, filename)
        tree = ET.parse(file_path)
        sentences = tree.findall("sentence")
        for sentence in sentences:
            (sid, stext) = (sentence.attrib["id"], sentence.attrib["text"])
            if not stext:
                continue
            entities = get_entities(sentence)            
            tokenized_sentence = tokenize(stext)
            dataset_dict[sid] = tokenized_sentence

    return dataset_dict

def create_indexs(datadir, max_length):
    '''
    Returns a mapping of each word seen in the data with an integer. Also a mapping of each tag (null, mechanism, advise,
    effect, int). Also returns maxlen
    It has an <UNK> token and <PAD> token that will be used for filling the encoded sentence till max_length
    '''
    tags = ['null', 'mechanism', 'advise', 'effect', 'int']
    all_indexes = {}
    word_indexes = {}
    tags_indexes = dict(enumerate(tags))
    all_indexes['maxlen'] = max_length
    all_indexes['tags'] = tags_indexes



def learn (traindir, validationdir, modelname):
    '''
    learns a NN model using traindir as training data , and validationdir
    as validation data . Saves learnt model in a file named modelname
    '''
    # load train and validation data in a suitable form
    traindata = load_data(traindir)
    valdata = load_data(validationdir)

    # create indexes from training data
    max_len = 100
    idx = create_indexs(traindata, max_len)

    # build network
    model = build_network(idx)

    # encode datasets
    Xtrain = encode_words(traindata,idx)
    Ytrain = encode_tags(traindata, idx)
    Xval = encode_words(valdata, idx)
    Yval = encode_tags(valdata, idx)

    # train model
    model.fit(Xtrain, Ytrain, validation_data=(Xval, Yval))

    # save model and indexs , for later use in prediction
    save_model_and_indexs(model, idx, modelname)


### MAIN ###
data_loaded = load_data("data/Train")


SyntaxError: invalid syntax (probability.py, line 333)